In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import os
import glob
import shutil

import cv2
import random
import pickle


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential,Model
from keras.layers import  Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.metrics import sparse_categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
DIRECTORY='/kaggle/input/herbs-plant-dataset/herbs classification'
print(DIRECTORY)
#print(os.listdir())
os.chdir("/kaggle/input/herbs-plant-dataset/herbs classification")
print(os.listdir())

In [ ]:
#train_path=DIRECTORY+'/train'
train_path='train'
valid_path='valid'
test_path='test'
print(test_path)
os.listdir(test_path)

In [ ]:
train_batches=ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input)\
.flow_from_directory(directory=train_path,target_size=(224,224),  batch_size=10)

#valid_batches=ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input)\
#.flow_from_directory(directory=valid_path,target_size=(224,224),  batch_size=10)

test_batches=ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input)\
.flow_from_directory(directory=test_path,target_size=(224,224),  batch_size=10,shuffle=False)

In [ ]:
vgg16_model = tf.keras.applications.vgg16.VGG16()
#model.summary()

model = Sequential()
for layer in vgg16_model.layers[:-2]:
    print(layer)
    model.add(layer)
    
for layer in model.layers[:-5]:
    #print(layer)
    layer.trainable = False

model.add(Dense(units=12, activation='softmax',  name='predictions'))

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.optimizers import Adam
model.compile(optimizer=SGD(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
hist=model.fit(x=train_batches, validation_data=test_batches, epochs=100,verbose=2)

In [ ]:
print(hist.history['loss'])


In [ ]:
#plot the training and validation accuracy at each epoch
acc = hist.history['accuracy']
val_acc = hist.history['val_accuracy']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'y', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
#plot the training and validation  loss at each epoch
loss = hist.history['loss']
val_loss = hist.history['val_loss']
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
test_labels=test_batches.classes
test_labels

In [ ]:
predict=model.predict(x=test_batches, verbose=0)
#print(predict[0])
print(predict.argmax(axis=1))

In [ ]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_true=test_labels, y_pred=predict.argmax(axis=1))

print(cm)

sns.heatmap(cm, annot=True)

In [ ]:
# Showing the Classification report 
from sklearn.metrics import classification_report
cr=classification_report(test_labels, predict.argmax(axis=1))
print(cr)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# Load ResNet-101 model without classification layers and specify input shape
base_model_resnet = ResNet101(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add your own classification layers
x = base_model_resnet.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(12, activation='softmax')(x)

# Create a new model combining the base ResNet-101 and the custom classification layers
model_resnet = Model(inputs=base_model_resnet.input, outputs=predictions)

# Freeze layers in the base model
for layer in base_model_resnet.layers:
    layer.trainable = False

model_resnet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history_resnet = model_resnet.fit(train_batches, validation_data=test_batches, epochs=100)# Create ResNet-101 model
resnet_model = ResNet101(weights='imagenet', include_top=False)
model_resnet = Sequential()
model_resnet.add(resnet_model)
model_resnet.add(Dense(units=12, activation='softmax', name='predictions'))

for layer in model_resnet.layers[0].layers[:-6]:
    layer.trainable = False

model_resnet.compile(optimizer=SGD(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# Load Xception model without classification layers and specify input shape
base_model_xception = Xception(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add your own classification layers
x = base_model_xception.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(12, activation='softmax')(x)

# Create a new model combining the base Xception and the custom classification layers
model_xception = Model(inputs=base_model_xception.input, outputs=predictions)

# Freeze layers in the base model
for layer in base_model_xception.layers:
    layer.trainable = False

model_xception.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history_xception = model_xception.fit(train_batches, validation_data=test_batches, epochs=100)


In [ ]:
# Plot training and validation accuracy
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history_xception.history['accuracy'], label='Training Accuracy')
plt.plot(history_xception.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Plot training and validation loss
plt.subplot(1, 2, 2)
plt.plot(history_xception.history['loss'], label='Training Loss')
plt.plot(history_xception.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:

# Evaluate on test set and generate confusion matrix
predictions =  model_xception.predict(test_batches)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_batches.classes
confusion_mat = confusion_matrix(true_classes, predicted_classes)
print("Confusion Matrix:")
print(confusion_mat)

# Plot confusion matrix
sns.heatmap(confusion_mat, annot=True)

# Display classification report
class_names = list(test_batches.class_indices.keys())
report = classification_report(true_classes, predicted_classes, target_names=class_names)
print("Classification Report:")
print(report)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, classification_report

DIRECTORY = '/kaggle/input/herbs-plant-dataset/herbs classification'
train_path = 'train'
test_path = 'test'

# Load the training and test datasets using ImageDataGenerator
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet.preprocess_input) \
    .flow_from_directory(directory=train_path, target_size=(224, 224), batch_size=10)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet.preprocess_input) \
    .flow_from_directory(directory=test_path, target_size=(224, 224), batch_size=10, shuffle=False)

# Load the pre-trained ResNet-101 model
resnet_model = tf.keras.applications.ResNet101(weights='imagenet', include_top=False)

# Freeze layers in the ResNet-101 model
for layer in resnet_model.layers:
    layer.trainable = False

# Create a new model by adding layers on top of the ResNet-101 base
model = Sequential([
    resnet_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    Dense(units=12, activation='softmax', name='predictions')
])

# Compile the model
model.compile(optimizer=SGD(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
hist = model.fit(x=train_batches, validation_data=test_batches, epochs=100, verbose=2)

# Plot training and validation accuracy
plt.plot(hist.history['accuracy'], 'y', label='Training acc')
plt.plot(hist.history['val_accuracy'], 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plot training and validation loss
plt.plot(hist.history['loss'], 'y', label='Training loss')
plt.plot(hist.history['val_loss'], 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Evaluate on test set and generate confusion matrix
predictions = model.predict(test_batches)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_batches.classes
confusion_mat = confusion_matrix(true_classes, predicted_classes)
print("Confusion Matrix:")
print(confusion_mat)

# Plot confusion matrix
sns.heatmap(confusion_mat, annot=True)

# Display classification report
class_names = list(test_batches.class_indices.keys())
report = classification_report(true_classes, predicted_classes, target_names=class_names)
print("Classification Report:")
print(report)
